### Student Information
Name: Duygu

Student ID: X1100011

GitHub ID: duygu-bayrak

Kaggle name: duygu

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

# 1. Part

In [15]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Code for Exercise 1

In [2]:
import pandas as pd
path = "/kaggle/input/dmlab2-ww/DM2021-Lab2-master/"
### training data
anger_train = pd.read_csv(path+"data/semeval/train/anger-ratings-0to1.train.txt",
                         sep="\t", header=None,names=["id", "text", "emotion", "intensity"])
sadness_train = pd.read_csv(path+"data/semeval/train/sadness-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_train = pd.read_csv(path+"data/semeval/train/fear-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_train = pd.read_csv(path+"data/semeval/train/joy-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

In [3]:
# combine 4 sub-dataset
train_df = pd.concat([anger_train, fear_train, joy_train, sadness_train], ignore_index=True)

In [4]:
### testing data
anger_test = pd.read_csv(path+"data/semeval/dev/anger-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
sadness_test = pd.read_csv(path+"data/semeval/dev/sadness-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_test = pd.read_csv(path+"data/semeval/dev/fear-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_test = pd.read_csv(path+"data/semeval/dev/joy-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

# combine 4 sub-dataset
test_df = pd.concat([anger_test, fear_test, joy_test, sadness_test], ignore_index=True)
train_df.head()

In [5]:
# shuffle dataset
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)

---
### ** >>> Exercise 1 (Take home): **  
Plot word frequency for Top 30 words in both train and test dataset. (Hint: refer to DM lab 1)


In [6]:
# Answer here
import re
from collections import Counter

def split_sentence(sentence):
    return list(filter(lambda x: len(x) > 0, re.split('\W+', sentence.lower())))

def generate_vocabulary(train_captions, min_threshold):
    """
    Return {token: index} for all train tokens (words) that occur min_threshold times or more, 
        `index` should be from 0 to N, where N is a number of unique tokens in the resulting dictionary.
    """  
    #convert the list of whole captions to one string
    concat_str = ' '.join([str(elem).strip('\n') for elem in train_captions]) 
    #divide the string tokens (individual words), by calling the split_sentence function 
    individual_words = split_sentence(concat_str)
    #create a list of words that happen min_threshold times or more in that string  
    #condition_keys = sorted([key for key, value in Counter(individual_words).items() if value >= min_threshold])
    #generate the vocabulary(dictionary)
    #result = dict(zip(condition_keys, range(len(condition_keys))))
    return Counter(individual_words)#result

In [7]:
# word frequency for Top 30 words in train dataset
train_freqs = generate_vocabulary(train_df.text.tolist(), 0)
train_freqs.most_common(30)

In [8]:
# word frequency for Top 30 words in train dataset
test_freqs = generate_vocabulary(test_df.text.tolist(), 0)
test_freqs.most_common(30)

---
### ** >>> Exercise 2 (Take home): **  
Generate an embedding using the TF-IDF vectorizer instead of th BOW one with 1000 features and show the feature names for features [100:110].

In [10]:
import nltk
nltk.download('punkt')

In [11]:
# Answer here
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000, tokenizer=nltk.word_tokenize)

train_data_tfidf_features_1000 = tfidf_vectorizer.fit_transform(train_df['text'])
test_data_tfidf_features = tfidf_vectorizer.transform(test_df['text'])

train_data_tfidf_features_1000.shape

In [12]:
feature_names_tfidf = tfidf_vectorizer.get_feature_names()
feature_names_tfidf[100:110]

In [13]:
"😂" in feature_names_tfidf

## code needed for Exercise 3

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_df['text'])

train_data_BOW_features_500 = BOW_500.transform(train_df['text'])

## check dimension
train_data_BOW_features_500.shape

In [21]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])
y_test = test_df['emotion']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

In [22]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [23]:
## check by confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

In [25]:
# Funciton for visualizing confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix',
                          cmap=sns.cubehelix_palette(as_cmap=True)):
    """
    This function is modified from: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    classes.sort()
    tick_marks = np.arange(len(classes))    
    
    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels = classes,
           yticklabels = classes,
           title = title,
           xlabel = 'True label',
           ylabel = 'Predicted label')

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    ylim_top = len(classes) - 0.5
    plt.ylim([ylim_top, -.5])
    plt.tight_layout()
    plt.show()

In [26]:
# plot your confusion matrix
my_tags = ['anger', 'fear', 'joy', 'sadness']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

---
### ** >>> Exercise 3 (Take home): **  
Can you interpret the results above? What do they mean?


# Answer here
the model classified:
- 57 sentences correctly as anger
- 75 sentences correctly as fear
- 53 sentences correctly as joy
- 44 sentences correctly as sadness

the model misclassified:
- 11 fear, 7 joy, and 9 sadness sentences as anger
- 11 anger, 14 joy, and 10 sadness sentences as fear
- 6 anger, 13, fear, and 7 sadness sentences as joy
- 7 anger, 12 fear, and 11 joy sentences as sadness

---
### ** >>> Exercise 4 (Take home): **  
Build a model using a ```Naive Bayes``` model and train it. What are the testing results? 

*Reference*: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html


In [27]:
# Answer here
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)

#train
clf.fit(X_train, y_train)

## predict!
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [28]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

---
### ** >>> Exercise 5 (Take home): **  

How do the results from the Naive Bayes model and the Decision Tree model compare? How do you interpret these differences? Use the theoretical background covered in class to try and explain these differences.

# Answer here
The decision tree model yielded a training accuracy of 0.99 and a testing accuracy of 0.66.
The naive bayes model yielded a training accuracy of 0.8 and a testing accuracy of 0.73.

Since the first model has a training accuracy of 99%, I would say that it is overfitting to the train data. This would also explain the lower testing accuracy compared to the second model.
The second model generalizes better, the testing accuracy is not much lower than the training accuracy. This is because the Naive Bayes classifier employs a very simple (linear) hypothesis function, which prevents it from overfitting to its training data.



## Code for Exercise 6

In [34]:
# pip3 install keras
import keras

In [35]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc) #keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)


In [36]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [37]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [41]:
# You can write up to 20GB to the current directory (/kaggle/working/)
path = "/kaggle/working/"

import pandas as pd
df = pd.DataFrame(list())
df.to_csv(path+'training_log.csv')

In [42]:
from keras.callbacks import CSVLogger


csv_logger = CSVLogger(path+'training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')

In [44]:
#Let's take a look at the training log
training_log = pd.DataFrame()
training_log = pd.read_csv(path+"training_log.csv")
training_log.head()

---
### ** >>> Exercise 6 (Take home): **  

Plot the Training and Validation Accuracy and Loss (different plots), just like the images below (Note: the pictures below are an example from a different model). How to interpret the graphs you got? How are they related to the concept of overfitting/underfitting covered in class?
<table><tr>
    <td><img src="pics/pic3.png" style="width: 300px;"/> </td>
    <td><img src="pics/pic4.png" style="width: 300px;"/> </td>
</tr></table>

In [45]:
# Answer here
train_acc = training_log['accuracy'].tolist()
val_acc = training_log['val_accuracy'].tolist()
train_loss = training_log['loss'].tolist()
val_loss = training_log['val_loss'].tolist()

In [46]:
from matplotlib import pyplot as plt

plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

plt.plot(train_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

we can clearly see an overfitiing here. If we look at the graph "model loss", we can see that the validation loss first decreases until epoch 4 and then it increases. The train loss, however, keeps decreasing and reaches almost 0% at epoch 25. If we then look at the graph "model accuracy", we can see that the train accuracy increases very steeply and flattens after epoch 5. So, at about epoch 4 the train accuracy reaches 90% and this is the point where the overfitting happens since the validation loss then starts to increase again.

---
### ** >>> Exercise 7 (Take home): **  

Now, we have the word vectors, but our input data is a sequence of words (or say sentence). 
How can we utilize these "word" vectors to represent the sentence data and train our model?


## Answer: we can calculate the average vector for all words in every sentence (see code below)

In [ ]:
# Answer here
# calculate the average vector for all words in every sentence
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.vocab]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

## Code for Exercise 8

---
### ** >>> Exercise 8 (Take home): **  

Generate a t-SNE visualization to show the 15 words most related to the words "angry", "happy", "sad", "fear" (60 words total).

In [47]:
from gensim.models import KeyedVectors

## Note: this model is very huge, this will take some time ...
model_path = "/kaggle/input/dmlab2-ww/GoogleNews-vectors-negative300.bin" #"GoogleNews/GoogleNews-vectors-negative300.bin.gz"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print('load ok')

w2v_google_model.most_similar('happy', topn=10)

In [48]:
# Answer here
word_list = ['happy', 'angry', 'sad', 'fear']

topn = 15
happy_words = [word_ for word_, sim_ in w2v_google_model.most_similar('happy', topn=topn)]
angry_words = [word_ for word_, sim_ in w2v_google_model.most_similar('angry', topn=topn)]        
sad_words = [word_ for word_, sim_ in w2v_google_model.most_similar('sad', topn=topn)]        
fear_words = [word_ for word_, sim_ in w2v_google_model.most_similar('fear', topn=topn)]        

print('happy_words: ', happy_words)
print('angry_words: ', angry_words)
print('sad_words: ', sad_words)
print('fear_words: ', fear_words)

target_words = happy_words + angry_words + sad_words + fear_words
print('\ntarget words: ')
print(target_words)

print('\ncolor list:')
cn = topn 
color = ['b'] * cn + ['g'] * cn + ['r'] * cn + ['y'] * cn
print(color)

In [50]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

## w2v model
model = w2v_google_model

## prepare training word vectors
size = 200
target_size = len(target_words)
all_word = list(model.index_to_key)
word_train = target_words + all_word[:size]
X_train = model[word_train]

## t-SNE model
tsne = TSNE(n_components=2, metric='cosine', random_state=28)

## training
X_tsne = tsne.fit_transform(X_train)

## plot the result
plt.figure(figsize=(7.5, 7.5), dpi=115)
plt.scatter(X_tsne[:target_size, 0], X_tsne[:target_size, 1], c=color)
for label, x, y in zip(target_words, X_tsne[:target_size, 0], X_tsne[:target_size, 1]):
    plt.annotate(label, xy=(x,y), xytext=(0,0),  textcoords='offset points')
plt.show()

# 3. Part

In [51]:
import zipfile
import pandas as pd
import json
from tqdm import tqdm

### 1. load data

In [ ]:
# /kaggle/input/dm2021-lab2-hw2/tweets_DM.json
# /kaggle/input/dm2021-lab2-hw2/sampleSubmission.csv
# /kaggle/input/dm2021-lab2-hw2/data_identification.csv
# /kaggle/input/dm2021-lab2-hw2/emotion.csv

In [52]:
X_id = pd.read_csv('/kaggle/input/dm2021-lab2-hw2/data_identification.csv')
X_id.head()
X_id["identification"].value_counts()

In [69]:
json_lines = []

with open('/kaggle/input/dm2021-lab2-hw2/tweets_DM.json', 'r') as f:
    for line in tqdm(f):
        tweet = json.loads(line)['_source']
        json_lines.append([tweet["tweet"]["tweet_id"], tweet["tweet"]["text"]])
        

X_text = pd.DataFrame(json_lines, columns =['tweet_id', 'text'])
X_text.head()

In [70]:
X_id_text = pd.merge(X_id, X_text, on = "tweet_id")
X_id_text.head()

In [79]:
X_id_text_train =  X_id_text[X_id_text['identification'] == "train"]
X_test_df =  X_id_text[X_id_text['identification'] == "test"]
X_test_df.head()

In [80]:
# reset index
X_test_df.reset_index(drop=True, inplace=True)
X_test_df.head()

In [72]:
X_emotion = pd.read_csv('/kaggle/input/dm2021-lab2-hw2/emotion.csv')
X_train_df = pd.merge(X_id_text_train, X_emotion, on = "tweet_id")
X_train_df.head()

In [81]:
path = "/kaggle/working/"

X_train_df.to_pickle(path+"train.pkl")
X_test_df.to_pickle(path+"test.pkl")

## 2. data visualization

In [60]:
import plotly.express as px
import matplotlib.pyplot as plt

In [61]:
fig=px.histogram(X_train_df,
                x='emotion',
                title='Sentiment Count ',
                color_discrete_sequence=['green'])
fig.update_layout(bargap=0.1)
fig.show()
fig = px.pie(X_train_df, names='emotion',title="Sentiment Distribution")
fig.show()

## 3. pre-processing

In [62]:
import re
import numpy as np

In [63]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec

In [93]:
import string

def data_cleaning(data):
    cleaned_data = []
    fillerWord = ("so","yeah","okay","um","uh","mmm","ahan","uh","huh","ahm","oh","sooo","uh","huh","yeh","yah","hmm","bye")
    fillerword_reg= "bye[.,]|so[.,]|yeah[.,]|okay[.,]|um[.,]|uh[.,]|mmm[.,]|ahan[.,]|uh[.,]|huh[.,]|ahm[.,]|oh[.,]|sooo[.,]|uh[.,]|huh[.,]|yeh[.,]|yah[.,]|hmm[.,]"
    STOPWORDS = set(stopwords.words('english'))
    remove=["doesn't","not","nor","neither","isn't","hadn't","mightn't","needn't","wasn't"]
    for i in remove:
        STOPWORDS.discard(i)
    
    STOPWORDS.add(fillerWord)  
    for i in range(len(data)):
        tweet = re.sub("#", "", data[i])#extracting hashtags
        tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '',tweet, flags=re.MULTILINE)#extracting links
        html=re.compile(r'<.*?>')#extracting html tags
        tweet =html.sub(r"", tweet)
        #extracting symbols and characters
        tweet=re.sub(r'@\w+',"",tweet)
        tweet=re.sub(r'#\w+',"",tweet) 
        tweet=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet) 
        punctuation = r"\"!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
        #tweet.rstrip(string.punctuation)
        tweet = re.sub(r"""
               [,.;@#?!&$]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               tweet)
        #tweet = re.sub(punctuation, " ", tweet)
        tweet=re.sub('[^A-Za-z\s]+',"", tweet)
        tweet = tweet.lower()
        tweet = tweet.split()
        #Lemmatization to normalise text
        tweet = [lemmatizer.lemmatize(word) for word in tweet if not word in STOPWORDS]
        tweet = ' '.join(tweet)
        filler=re.compile(fillerword_reg)
        tweet=filler.sub("",tweet)
        cleaned_data.append(tweet)
    return cleaned_data

In [94]:
####Preprocessing and cleaning train data
X_train_df['OriginalLength']= X_train_df['text'].apply(lambda x: len(x.split(" ")))
texto_data=X_train_df['text'].tolist()
process_text=data_cleaning(texto_data)

cleaned_tweet =pd.DataFrame(process_text)
X_train_df['CleanedTweet']=cleaned_tweet

#Adding length of cleaned tweet to dataset
X_train_df['NewLength']= X_train_df['CleanedTweet'].apply(lambda x:len(x.split(" ")))

X_train_df.head()

In [95]:
####Preprocessing and cleaning test data
X_test_df['OriginalLength']= X_test_df['text'].apply(lambda x: len(str(x).split(" ")))
texto_data=X_test_df['text'].tolist()
process_text=data_cleaning(texto_data)

cleaned_tweet =pd.DataFrame(process_text)
X_test_df['CleanedTweet'] = cleaned_tweet

X_test_df['NewLength']= X_test_df['CleanedTweet'].apply(lambda x:len(str(x).split(" ")))

X_test_df.head()

In [96]:
path = "/kaggle/working/"

X_train_df.to_pickle(path+"train.pkl")
X_test_df.to_pickle(path+"test.pkl")

## 4. Feature Extraction (using TF-IDF)

In [ ]:
# # Answer here
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer(max_features=1000, tokenizer=nltk.word_tokenize)

# train_data_tfidf_features_1000 = tfidf_vectorizer.fit_transform(X_train_df['text'])
# test_data_tfidf_features = tfidf_vectorizer.transform(X_test_df['text'])

# train_data_tfidf_features_1000.shape

In [97]:
X_train = X_train_df.CleanedTweet
y_train = X_train_df.emotion

In [98]:
# encode labels
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_train[:5]

In [99]:
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
vectorizer = TfidfVectorizer()

X_train_data = vectorizer.fit_transform(X_train)
X_train_data.shape

## 5. Model training

In [101]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_train_data, y_train, test_size=0.33,random_state = 15)

In [102]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [103]:
model1_nb = MultinomialNB(alpha=0.1)
y_train_pred = model1_nb.fit(X1_train,y1_train).predict(X1_train)

y_pred = model1_nb.predict(X1_test)
  
# comparing actual response values  with predicted response values 
from sklearn import metrics
print("Multinomial Naive Bayes model train accuracy(in %):", metrics.accuracy_score(y1_train, y_train_pred)*100)
print("Multinomial Naive Bayes model test accuracy(in %):", metrics.accuracy_score(y1_test, y_pred)*100)


In [104]:
#Hyper parameter tuning
from sklearn.model_selection import GridSearchCV

In [105]:
params = {'alpha': [0.01,0.1,0.5,1,10],
         }

multinomial_nb_grid = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
multinomial_nb_grid.fit(X_train_data,y_train)

print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X1_train, y1_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X1_test, y1_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)


## 6. Predict on test data

In [106]:
test_ids = X_test_df["tweet_id"].to_list()

In [117]:
X_test = X_test_df.CleanedTweet.tolist()
X_test_data = vectorizer.transform(X_test)
X_test_data.shape

In [119]:
y_test_pred = model1_nb.predict(X_test_data)

In [127]:
y_test_pred = encoder.inverse_transform(y_test_pred)
y_test_pred[:20]

In [128]:
result = []
for i, id in enumerate(test_ids):
    result.append([id, y_test_pred[i]])

In [129]:
result[0]

In [130]:
import csv

header = ['id', 'emotion']

path = "/kaggle/working/"
with open(path+'pred.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(result)